In [1]:
%pylab
import wradlib
import cv2
import numpy as np
import os
from scipy import stats
from matplotlib import animation
import matplotlib.patches as mpatches
matplotlib.collections import PatchCollection
from scipy.ndimage import zoom
import datetime
import warnings
warnings.simplefilter('once', DeprecationWarning)

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


## Read sample data

Data is from the German Weather Service: the so called RY product represents rainfall intensity composite for the whole of Germany in 5 minute intervals. 

Spatial resolution: `1 x 1 km`; spatial extent: `900 x 900 km`.

**Information required from user**

- specify the directory `datadir` where you store the RY data (unpack the ry archives there).
- select a specific interval by commenting/uncommenting the `dtimes` lines.
- decide whether you need to reduce the resolution (downsize the image by a `downsizeby`) in order to avoid memory problems

In [2]:
# Set data directory
datadir = "data/ry"

# Original grid dimensions
nx = 900
ny = 900

# pixel size (in meters)
dx = 1000.
dy = 1000.

# Downsize by factor "downsizeby"
#    downsizeby = 1 will leave the dimensions unchanged,
#    but for a 900x900 km grid, downsizing might be 
#    required in order to avoid MemoryError
downsizeby = 1

# interval between observations (in seconds)
interval = 300

# Set time window
##dtimes = wradlib.util.from_to("2008-06-02 17:00:00", "2008-06-02 19:00:00", interval)
##dtimes = wradlib.util.from_to("2015-04-26 17:00:00", "2015-04-26 19:00:00", interval)
##dtimes = wradlib.util.from_to("2015-03-29 17:00:00", "2015-03-29 19:00:00", interval)
dtimes = wradlib.util.from_to("2016-05-29 16:00:00", "2016-05-29 19:00:00", interval)

In [3]:
# Compute grid dimensions and grid coordinates after resampling
dx2, dy2 = dx*downsizeby, dy*downsizeby
nx2, ny2 = nx/downsizeby, ny/downsizeby

X2, Y2 = np.meshgrid( np.arange(0,nx2*dx2, dx2), np.arange(0,ny2*dy2, dy2) )

# Define container
frames = np.zeros( (len(dtimes), nx2, ny2 ) )

# Read the data, convert back to dBZ, and downsize
for i, dtime in enumerate(dtimes):
    fname = dtime.strftime( os.path.join(datadir, "raa01-ry_10000-%y%m%d%H%M-dwd---bin") )
    frames[i] = zoom( wradlib.io.read_RADOLAN_composite(fname, missing=0)[0], 1./downsizeby, order=1)
    frames[i] = wradlib.trafo.decibel( wradlib.zr.r2z(frames[i]) )
    frames[i][frames[i]<0] = 0 

e:\src\git\wradlib\wradlib\wradlib\trafo.py:127: RuntimeWarning: divide by zero encountered in log10
  return 10. * np.log10(x)


## Use OpenCV's Optical Flow to detect and track features

This example uses the Lucas-Kanade Optical Flow implementation in OpenCV (see [here](http://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_video/py_lucas_kanade/py_lucas_kanade.html)). We take the first frame, detect some Shi-Tomasi corner points in it, then we iteratively track those points over the subsequnet images.

The parameter dictionaries are certainly something to experiment with.

In [68]:
# FEATURE DETECTION: Parameters for ShiTomasi corner detection
feature_params = dict( maxCorners = 200,
                       qualityLevel = 0.2,
                       minDistance = 7,
                       blockSize = 21 )

# FEATURE TRACKING: Parameters for Lucas Kanade (lk) Optical Flow technique
lk_params = dict( winSize  = (20,20),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0))

# Over which time steps (of the data we've read in) do you want to track
trackstart = 0
trackend = 24

In [69]:
# Our approach requires 8 bit integers - so we need to normalize our radar data accordingly
#   (there might be a more elegant solution...)
minval = 0
maxval = 59 # dBZ in this case
iframes = frames.copy()
iframes[iframes<minval] = minval
iframes[iframes>maxval] = maxval
iframes = ((iframes / maxval)*255).astype(np.uint8)

In [70]:
# Find good features to track...
old = cv2.goodFeaturesToTrack(iframes[trackstart], mask = None, **feature_params)
print "Found %d good features to track." % len(old)

# Set containers to collect results (time steps in rows, corners in columns)
#   Tracking status
sts = np.zeros((trackend,len(old)), dtype=np.bool)
sts[0] = True
#   corner x coords
x = np.zeros((trackend,len(old))) * np.nan
x[0] = old[:,0,0]
#   corner y coords
y = np.zeros((trackend,len(old))) * np.nan
y[0] = old[:,0,1]
#   tracking error
errs = np.zeros((trackend,len(old))) * np.nan
errs[0] = 0.
#   Assign persistent corner IDs
ids = np.arange(len(old))

Found 130 good features to track.


In [71]:
# Track good features
for i in range(trackstart+1, trackend):
    # track current corners in next image
    new, st, err = cv2.calcOpticalFlowPyrLK(prevImg=iframes[i], nextImg=iframes[i+1], prevPts=old, nextPts=None, **lk_params)
    success = st.ravel()==1
    ids = ids[success]
    sts[i, ids] = True
    x[i, ids] = new[success,0,0]
    y[i, ids] = new[success,0,1]
    errs[i, ids] = err.ravel()[success]
    #sts = np.vstack( (sts, st.reshape(1,-1).astype(np.bool)))
    #cornerx = np.vstack( (cornerx,new[:,0,0].reshape(1,-1)) )
    #cornery = np.vstack( (cornery,new[:,0,1].reshape(1,-1)) )
    # new corners will be old in the next loop
    old = new[success]

# Incremental euclidic distance from starting point
trackdist = np.diff( np.sqrt( (x-x[0].reshape((1,-1)))**2 + (y-y[0].reshape((1,-1)))**2 ), axis=0 )
trackdist = np.vstack( (np.zeros((1,trackdist.shape[1])), trackdist))

# Plot feature persistence
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(211)
cb = plt.imshow(errs, interpolation="none", cmap="summer", vmax = 15)
plt.xlabel("Feature ID")
plt.ylabel("Tracking time step")
plt.colorbar(cb, shrink=0.5)
plt.title("Tracking error")

# Plot consistence of movement
ax = fig.add_subplot(212)
cb = plt.imshow(trackdist, interpolation="none", cmap="bwr", vmin=-5, vmax=5)
plt.xlabel("Feature ID")
plt.ylabel("Tracking time step")
plt.colorbar(cb, shrink=0.5)
plt.title("Incremental euclidian distance from starting point")

plt.tight_layout()

In [72]:
# Find good tracks (but what is a "good" track...?)
goodtrack = np.zeros(x.shape[1], dtype=np.bool)
for i in range(len(goodtrack)):
    # persistence of the track
    if len(np.where(sts[:,i])[0]) < 2:
        continue
    # consistency of movement
    if len(np.where(trackdist[:,i]<0)[0]) > 0:
        continue
    # tracking error
    if len(np.where(errs[:,i]>15)[0]) > 5:
        continue
    goodtrack[i] = True
print "Found %d good tracks and %d bad tracks." % (len(np.where(goodtrack)[0]), len(goodtrack)-len(np.where(goodtrack)[0])) 

Found 89 good tracks and 41 bad tracks.


C:\Anaconda3\envs\wradlib09\lib\site-packages\ipykernel\__main__.py:8: RuntimeWarning: invalid value encountered in less
C:\Anaconda3\envs\wradlib09\lib\site-packages\ipykernel\__main__.py:11: RuntimeWarning: invalid value encountered in greater


In [76]:
# Visualize tracks: white: good track, red: bad track
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, aspect="equal")
# average reflectivity as background image
ax.imshow(np.mean(frames[trackstart:trackend], axis=0), origin="lower", cmap="gray", interpolation="none")
plt.xlabel("Easting (# pixels)")
plt.ylabel("Northing (# pixels)")
plt.title("[Zoom in to inspect track properties]")
plt.grid(color="white")
plt.xlim(0,nx/downsizeby)
plt.ylim(0,nx/downsizeby)
for i, isgood in enumerate(goodtrack):
    ix = sts[:,i]
    color = "red"
    if isgood:
        color = "green"
    ax.plot(x[ix,i], y[ix,i],marker="None", color=color, markersize=14, linestyle="-")
    ax.arrow(x[ix,i][-2], y[ix,i][-2],
             np.diff(x[ix,i][-2:])[0], np.diff(y[ix,i][-2:])[0], 
             head_width=2, head_length=2, fc=color, ec=color)

### Experiment with feature detection and description

We now im at [features](http://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_feature2d/py_table_of_contents_feature2d/py_table_of_contents_feature2d.html) instead of corners: detection, description, and tracking

In [107]:
# SIFT
sift = cv2.xfeatures2d.SIFT_create()

kp = sift.detect(iframes[0],None)
print "Found %d keypoints." % len(kp)

Found 1059 keypoints.


In [96]:
from matplotlib.collections import PatchCollection

In [115]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, aspect="equal")
# average reflectivity as background image
ax.imshow(frames[0], origin="lower", cmap="gray", interpolation="none")
plt.xlabel("Easting (# pixels)")
plt.ylabel("Northing (# pixels)")
plt.title("[Zoom in to inspect feature properties]")
plt.grid(color="white")
plt.xlim(0,nx/downsizeby)
plt.ylim(0,nx/downsizeby)
patches = []
for kp_ in kp:
    if kp_.size > 5:
        circle = mpatches.Circle(kp_.pt, kp_.size, fill=False, edgecolor=color)
    #ax.add_patch(circle)
    patches.append(circle)
collection = PatchCollection(patches, facecolor="none", edgecolor=color)
ax.add_collection(collection)